# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("API_KEY_MAPBOX")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [63]:
# Getting the data from the top 10 expensive neighbourhoods
average_house_value_by_neighborhood = to_data.loc[:, ["neighbourhood","average_house_value"]].groupby(by=["neighbourhood"]).mean()
average_house_value_by_neighborhood_top10 = average_house_value_by_neighborhood.sort_values('average_house_value', ascending=False).head(10)
avg_house_price_by_neighbourhood =  to_data.loc[:, ["neighbourhood","average_house_value"]].groupby(by=["neighbourhood", "year"]).mean()

# Calculate the mean number of dwelling types units per year
to_data_filtered_grouped = to_data.loc[:, "single_detached_house":"other_house"].groupby(["year"]).sum()

# Calculate the average monthly shelter costs for owned and rented dwellings
to_data_owned_rented_grouped = to_data[["shelter_costs_owned","shelter_costs_rented"]].groupby(["year"]).mean()

# Create new series that contains the average_house_value per year
average_house_value = to_data["average_house_value"].groupby(["year"]).mean()

### Panel Visualization Functions

In [86]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
    
    # Load neighbourhoods coordinates data
    file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
    df_neighbourhood_locations = pd.read_csv(file_path)
    df_neighbourhood_locations.set_index("neighbourhood",inplace=True,drop=True)
    df_neighbourhood_locations
    
    # Join the average values with the neighbourhood locations
    df_neighbourhood_avg_house_value_locations = pd.concat([average_house_value_by_neighborhood, df_neighbourhood_locations], axis="columns", join="inner")

    # Create a scatter mapbox to analyze neighbourhood info
    map_fig = px.scatter_mapbox(df_neighbourhood_avg_house_value_locations, 
                        lat="lat", 
                        lon="lon",     
                        color="average_house_value", 
                        size="average_house_value",
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        height=600,
                        size_max=15, 
                        zoom=9)
    return map_fig

    
def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    barplot = data.hvplot.bar(
    title= title,
    xlabel = xlabel, 
    ylabel = ylabel,
    color = color
    )
    return barplot
    
def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    lineplot = data.hvplot.line(
    title= title,
    xlabel = xlabel, 
    ylabel = ylabel,
    color = color
    )
    return lineplot 

def average_house_value():
    """Average house values per year."""
    # Create and interactive hvplot for average_house_value per year for each neighbourhood


def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    # Create and interactive hvplot for average_house_value per year for each neighbourhood
    # Create new dataframe that contains the average_house_value per year for each neighbourhood
    
    avgvbyn_fig = avg_house_price_by_neighbourhood.hvplot(
        x='year', 
        y='average_house_value', 
        xlabel = 'Year', 
        ylabel = 'Average House Value',
        yformatter="%.0f",
        groupby="neighbourhood",
        widget_location='right_top'
    )
    return avgvbyn_fig

def number_dwelling_types():
    """Number of dwelling types per year"""
    # Calculate the sum number of dwelling types units per year (hint: use groupby)
    # Create new dataframe that contains the sum of each dwelling type by year 
    to_data_filtered_grouped = to_data.loc[:, "single_detached_house":"other_house"].groupby(["year"]).sum()
    return to_data_filtered_grouped

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    avg_house_snap_fig = create_line_chart(
    data = average_house_value,
    title = "Average House Value in Toronto",
    xlabel = "Year",
    ylabel = "Avg House Value",
    color = "blue"
    )
    return avg_house_snap_fig


def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    # Plotting the data from the top 10 expensive neighbourhoods
    top10_fig = average_house_value_by_neighborhood_top10.hvplot.bar(
        title="Top 10 Most Expensive Neighbourhoods in Toronto",
        x='neighbourhood', 
        y='average_house_value',
        rot=90,
        xlabel = 'Neighbourhood', 
        ylabel = 'Avg House Value',
        yformatter="%.0f",
        height=500,
        width=600
    )
    return top10_fig

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    # Fetch the data from all expensive neighbourhoods per year.
    # Create a list of the top 10 most expensive neighbourhoods as determined in the average_house_value_by_neighborhood_top10 section
    average_house_value_by_neighborhood_top10_list = average_house_value_by_neighborhood_top10.index.values.tolist()

    # Create a new dataframe that has shelter_costs_owned per year for each neighbourhood
    to_data_shelter_cost_owned = to_data[["neighbourhood","shelter_costs_owned"]].groupby(["neighbourhood","year"]).mean()

    # To be able to use this data frame for the sunburst plot we will first reset the index of this data frame then plot
    # The index reset is required as you are not able to pass multi indexed data to plotly.express 
    to_data_shelter_cost_owned_reset = to_data_shelter_cost_owned.reset_index()

    # Create a new dataframe that contains only neighbouhoods that are in the average_house_value_by_neighborhood_top10_list
    to_data_shelter_cost_owned_top10_neighbourhoods = to_data_shelter_cost_owned_reset.loc[to_data_shelter_cost_owned_reset["neighbourhood"].isin(average_house_value_by_neighborhood_top10_list)]
    to_data_shelter_cost_owned_top10_neighbourhoods.head()
    
    # Create the sunburst chart using the to_data_shelter_cost_owned_top10_neighbourhoods data frame 
    sb_fig = px.sunburst(
        to_data_shelter_cost_owned_top10_neighbourhoods, 
        title="Costs Analysis of Most Expensive Neighbourhoods in Toronto per Year ",
        path=["year","neighbourhood"],
        values="shelter_costs_owned",
        color_continuous_scale=px.colors.sequential.Blues,
        height = 600,
        width = 700,
        color="shelter_costs_owned",
            labels={
                "shelter_costs_owned": "Shelter Costs",
                }
    )
    sb_fig.show()

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
# YOUR CODE HERE!

# Define a welcome text
# YOUR CODE HERE!

# Create a tab layout for the dashboard
# YOUR CODE HERE!


# Create the main dashboard
# YOUR CODE HERE!

## Serve the Panel Dashboard

In [ ]:
# dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [83]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(to_data_filtered_grouped.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(to_data_filtered_grouped.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(to_data_filtered_grouped.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(to_data_filtered_grouped.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [84]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(to_data_owned_rented_grouped["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(to_data_owned_rented_grouped["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [65]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [87]:
average_house_value_snapshot()

AttributeError: 'function' object has no attribute 'hvplot'

In [56]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()